# Import

In [ ]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from main import backgammon

# Import datasets

In [ ]:
def load_dataset(file_path):
    # Load CSV using pandas
    df = pd.read_csv(file_path)
    
    # Split into features and labels
    data = df.values
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)  # Ensure shape (N, 1)
    
    return TensorDataset(X, y)

train_file = os.path.join("..","Data","Deep","BoardEquity",'train.txt')
validation_file = os.path.join("..","Data","Deep","BoardEquity",'validation.txt')
test_file = os.path.join("..","Data","Deep","BoardEquity",'train.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Example: iterate over a batch from the train set
for X_batch, y_batch in train_loader:
    print("Input shape:", X_batch.shape)
    print("Labels shape:", y_batch.shape)
    break

In [ ]:
## BGNET HERE ##

## Training loop

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assume: model is your instance of BGNet
model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training settings
num_epochs = 100  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch[:, :289], X_batch[:, 289:])
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * y_batch.size(0)

            preds = (outputs > 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# === Training Loop ===
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        board_left = X_batch[:, :289]
        board_right = X_batch[:, 289:]

        outputs = model(board_left, board_right)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * y_batch.size(0)

        # For training accuracy
        preds = (outputs > 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    avg_train_loss = running_loss / total
    train_acc = correct / total

    if epoch % 10 == 0:
        val_loss, val_acc = evaluate(model, val_loader)
        test_loss, test_acc = evaluate(model, test_loader)
        
        print(f"Epoch {epoch:03d}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")
        print(f"  Test  Loss: {test_loss:.4f}, Accuracy: {test_acc*100:.2f}%\n")

    torch.save(model.state_dict(), "backgammon_model.pth")
